In [1]:
from bs4 import BeautifulSoup
from random import randint, shuffle
from time import sleep
import re
import pandas as pd
import json
import math
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import os
from tqdm import tqdm_notebook as tqdm
import shutil
import numpy as np
import pandas_profiling
import pickle
from IPython.display import Image 

pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", -1)
pd.set_option("display.float_format", "{:,.4f}".format)
pd.set_option("display.max_info_rows", 200)


import surprise
from surprise.model_selection import train_test_split
from surprise import NormalPredictor
from surprise import Dataset
from surprise import Reader
from surprise import accuracy
from surprise.model_selection import cross_validate
from surprise.prediction_algorithms import SVD, SVDpp, BaselineOnly, NMF, CoClustering
from surprise.prediction_algorithms import KNNWithMeans, KNNBasic, KNNBaseline
from surprise.model_selection import GridSearchCV

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("white")
sns.set_context("paper", font_scale=1.8)
sns.set_palette("hls", 15)

# Dataset

In [2]:
# List of all products from Sephora as of June 24 with binary columns for categories

products = pd.read_csv("df_sephora.csv")

In [3]:
products.info(null_counts=True, max_cols=200)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9007 entries, 0 to 9006
Data columns (total 192 columns):
id                                   9007 non-null object
name                                 9007 non-null object
brand_id                             9007 non-null int64
brand_name                           9007 non-null object
short_desc                           8997 non-null object
long_desc                            8997 non-null object
item                                 9007 non-null int64
item_name                            7696 non-null object
list_price                           9007 non-null float64
variation                            9007 non-null object
variants                             3930 non-null object
price_low                            9007 non-null float64
price_high                           9007 non-null float64
is_limited_edition                   9007 non-null int64
rating                               8792 non-null float64
review_count         

In [34]:
df = pd.read_csv("user_item_rating_raw.csv")
df = df[["uid", "author_id", "product_id", "rating"]].copy()

/Applications/anaconda3/envs/learn-env/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (1,18,20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [35]:
author_ids = list(df.uid.unique())
product_ids = list(df.product_id.unique())

In [36]:
del df

In [40]:
selected_products = products[products.id.isin(product_ids)]
selected_products.shape

(6839, 192)

In [41]:
selected_products.to_csv("df_sephora_selected.csv", index=False)

In [9]:
products_short = products[products.id.isin(product_ids)][[
                        "id", "rating", "review_count", "categories"
                    ]].copy().sort_values(["rating", "review_count"], ascending=False)

products_short.shape

(6839, 4)

In [145]:
for_all = list(products[(products.gifts_her == 1) | (products.gifts_men == 1) |
         (products.gifts_them == 1) | (products.luxury_gifts == 1) |
         (products.editors_picks == 1) | (products.gift_sets == 1)].id.unique())

pickle.dump(for_all, open("for_all.pkl", "wb"))

In [125]:
for_men = list(products[(products.for_men==1)].id.unique())
pickle.dump(for_men, open("for_men.pkl", "wb"))

In [68]:
skincare = list(products_short[(products_short.categories.str.contains("cat150006", na=False))].id.unique())
pickle.dump(skincare, open("skincare.pkl", "wb"))

In [69]:
fragrance = list(products_short[(products_short.categories.str.contains("cat160006", na=False))].id.unique())
pickle.dump(fragrance, open("fragrance.pkl", "wb"))

In [70]:
makeup = list(products_short[(products_short.categories.str.contains("cat140006", na=False))].id.unique())
pickle.dump(makeup, open("makeup.pkl", "wb"))

In [147]:
hair_body = list(products_short[(products_short.categories.str.contains("cat130038|cat140014", na=False))].id.unique())
pickle.dump(hair_body, open("hair_body.pkl", "wb"))

# Model 1

In [47]:
rel = str(input("Relationship to recipient: ")).lower()

gender = str(input("He, She or They:  (enter 0 if unknown)")).lower()

budget_max = float(input("Maximum budget in US$: "))
budget_min = float(input("Minimum in US$: "))

age = int(input("Age: (please provide estimate if unknown): "))
stone = str(input("Skintone: (enter 0 if unknown):")).lower()
stype = str(input("Primary skintype: (enter 0 if unknown):")).lower()
sconcerns = str(input("Primary skinconcern: (enter 0 if unknown):")).lower()
ecolor = str(input("Eye color: (enter 0 if unknown):")).lower()
hcolor = str(input("Hair color: (enter 0 if unknown):")).lower()
hcondition = str(
    input("Primary hair condition: (enter 0 if unknown):")).lower()
hconcerns = str(input("Primary hair concern: (enter 0 if unknown):")).lower()


if gender in ["they", "0"]:
    cats = ["0"]
elif rel in ["friend", "dad", "brother", "son"] and gender=="he":
    cats = ["0"]
elif rel in ["client", "boss", "colleague"] and gender in ["he", "she"]:
    cats = ["0"]
else:
    cats = str(
        input("What time of products? Skincare, Makeup, Fragrance: (separate options using comma ',')")).replace(
            " ", "").replace("-", "").lower().split(",")
    
top_ = get_reco(age=age,
     stone = stone,
     stype = stype,
     sconcerns = sconcerns,
     ecolor = ecolor,
     hcolor = hcolor,
     hcondition = hcondition,
     hconcerns = hcondition,
     cats = cats,
     rel = rel,
     gender = gender,
     budget_max = budget_max,
     budget_min = budget_min,
     n=15)

In [49]:
top_

,iid,est,id,name,brand_id,brand_name,short_desc,long_desc,item,item_name,list_price,variation,variants,price_low,price_high,is_limited_edition,rating,review_count,user_favorites,categories,url,image_url,ingredients,suggested_usage,brand_long,has_variants,has_categories,has_ingredients,has_suggested_usage,is_rated,cologne,men,fragrance,perfume,women,lotions_and_oils,rollerballs_and_travel_size,perfume_gift_sets,value_and_gift_sets,lipstick,lip,makeup,lip_balms_and_treatments,lip_treatments,mascara,eye,lip_gloss,eyebrow,body_sprays_and_deodorant,bath_and_shower,cologne_gift_sets,body_mist_and_hair_mist,concealer,face,lip_stain,deodorant_and_antiperspirant,body_care,bath_and_body,face_primer,foundation,tinted_moisturizer,face_serums,treatments,skincare,mists_and_essences,moisturizers,highlighter,liquid_lipstick,lip_sets,face_sets,setting_spray_and_powder,aftershave,shaving,eye_creams_and_treatments,eye_care,face_masks,masks,face_wash_and_cleansers,cleansers,exfoliators,makeup_removers,eyeliner,face_brushes,makeup_brushes_and_applicators,makeup_palettes,eye_palettes,bb_and_cc_creams,blush,cheek,facial_peels,eyelash_curlers,accessories,eye_sets,mini_size,bronzer,face_oils,contour,lip_liner,eyeshadow,face_wash,moisturizer_and_treatments,toners,body_lotions_and_body_oils,body_moisturizers,scrub_and_exfoliants,self_tanners,face_sunscreen,sun_care,blemish_and_acne_treatments,facial_cleansing_brushes,high_tech_tools,for_face,color_correct,skincare_sets,decollete_and_neck_creams,hand_cream_and_foot_cream,candles_and_home_scents,night_creams,sheet_masks,eye_brushes,beauty_supplements,wellness,facial_rollers,body_sunscreen,cellulite_and_stretch_marks,for_body,body_wash_and_shower_gel,nail,other_needs,shampoo_and_conditioner,hair,hair_styling_and_treatments,conditioner,shampoo,eye_masks,eye_primer,mirrors_and_sharpeners,tools_and_brushes,cheek_palettes,face_wipes,tweezers_and_eyebrow_tools,hair_accessories,hair_tools,gift_cards,false_eyelashes,brush_sets,brush_cleaners,makeup_and_travel_cases,hair_removal,lip_brushes,sponges_and_applicators,makeup_bags_and_travel_cases,hair_masks,curling_irons,hair_straighteners_and_flat_irons,hair_dryers,dry_shampoo,hair_brushes_and_combs,scalp_and_hair_treatments,bath_soaks_and_bubble_bath,hair_styling_products,blotting_papers,hair_primers,hair_oil,teeth_whitening,anti_aging,travel_size,facial_men,candles,diffusers,holistic_wellness,leave_in_conditioner,hair_spray,color_care,best_for,just_arrived,gifts_her,gifts_men,gifts_them,gifts_teens,gift_sets_men,gift_sets,editors_picks,luxury_gifts,sephora_exclusives,perfume_men,grooming_men,hair_men,personal_care_men,just_arrived_men,best_seller_men,vegan,cruelty_free,for_men
0,P459128,4.9206,P459128,Vitamin C+B+E Ferulic Serum,6318,Dr. Lara Devgan Scientific Beauty,A powerful antioxidant that helps reduce the look of wrinkles and hyperpigmentation and supports natural collagen.,"What it is: A powerful antioxidant that helps reduce the look of wrinkles and hyperpigmentation and supports natural collagen. Skin Type: Normal, Dry, Combination, and Oily Skincare Concerns: Loss of Firmness and Elasticity, Dark Spots, and Fine Lines and Wrinkles Formulation: Lightweight Serum Highlighted Ingredients: - High-Potency Vitamin C: Fades the appearance of brown spots and fine lines. - Vitamin E: Soothes skin and reduces the appearance of fine lines. - Ferulic Acid: Helps protect the skin from UV rays. Ingredient Callouts: Free of sulfates SLS and SLES, parabens, formaldehydes, formaldehyde-releasing agents, phthalates, mineral oil, retinyl palmitate, oxybenzone, coal tar, hydroquinone, triclosan, and triclocarban, and contains less than one percent of synthetic fragrances. This product is also vegan, cruelty-free, and gluten-free, and comes in recyclable packaging. What Else You Need to Know: The Vitamin C+B+E Ferulic Serum is an advanced formulation that contains vitamins C, B, and E combined with ferulic acid and mixed-molecular-weight hyaluronic acid. This all-in-one serum

In [46]:
def get_reco(age,
             stone,
             stype,
             sconcerns,
             ecolor,
             hcolor,
             hcondition,
             hconcerns,
             cats,
             rel,
             gender,
             budget_max = 1000,
             budget_min = 0,
             n=50):
    
    if budget_min>budget_max:
        return print("Error: min budget is greater than max budget")
    
    else:
    
        products_to_consider = get_products(cats, rel, gender)

        if len(products_to_consider)== 0 or type(products_to_consider)==str:
            return print("Error: reenter details")
        else:
            uid_query = get_uid(age, stone, stype, sconcerns, ecolor, hcolor,
                                hcondition, hconcerns)

            if uid_query=="error":
                return print("Error: reenter details")
            else:
                products = pd.read_csv("df_sephora_selected.csv")
                topn = get_topn(uid_query, products_to_consider, n=len(products))
                topn = topn.merge(products, how="left", left_on="iid", right_on="id")
                topn = topn[(topn.price_high <= budget_max)
                            & (topn.price_high >= budget_min)].reset_index(
                                drop=True).head(n)
    return topn

In [7]:
def show_image(product_id):
    image_folder_path = "/Users/valmadrid/DataScienceBootcamp/Projects/Gift Recommendation/Gift-Recommendation/image/"
    display(Image(image_folder_path + product_id + ".png", width=250, height=250))

In [ ]:
def age_range(age):

    age_dict = {
        "13to17": range(1, 17 + 1),
        "18to24": range(18, 24 + 1),
        "25to34": range(25, 34 + 1),
        "35to44": range(35, 44 + 1),
        "45to54": range(45, 54 + 1),
        "over54": range(54, 100 + 1),
        "0": [0]
    }

    if age > 100 or age < 0:
        print("please check age input")
        return "error"

    for key, val in age_dict.items():
        if age in val:
            age_group = key
            if age < 13:
                print("products may not be aged appropriate")
            return age_group


def get_uid(age="0",
            stone="0",
            stype="0",
            sconcerns="0",
            ecolor="0",
            hcolor="0",
            hcondition="0",
            hconcerns="0"):

    ager = age_range(age)
    if ager == "error":
        return "error"

    skintone_options = [
        "0", 'porcelain', 'fair', 'light', 'medium', 'tan', 'olive', 'dark',
        'deep', 'ebony'
    ]

    skintype_options = ["0", "normal", "oily", "dry", "combination"]

    sconcerns_options1 = [
        "0", "acne", "aging", "blackheads", "uneven skintones", "sensitivity",
        "dark circles", "calluses", "pores", "redness", "dullness",
        "cellulite", "sun damage", "cuticles", "stretch marks", "puffiness"
    ]

    sconcerns_options2 = [
        "0", "acne", "aging", "blackheads", "unevenSkinTones", "sensitivity",
        "darkCircles", "calluses", "pores", "redness", "dullness", "cellulite",
        "sunDamage", "cuticles", "stretchMarks", "puffiness"
    ]

    ecolor_options = ["0", 'green', 'blue', 'brown', 'hazel', 'gray']

    hcolor_options = [
        "0", 'brunette', 'black', 'red', 'blonde', 'gray', 'auburn'
    ]

    hcondition_options1 = [
        "0", 'chemically treated', 'normal', 'wavy', 'fine', 'curly', 'coarse',
        'dry', 'oily', 'straight'
    ]

    hcondition_options2 = [
        "0", 'chemicallyTreated', 'normal', 'wavy', 'fine', 'curly', 'coarse',
        'dry', 'oily', 'straight'
    ]

    hconcerns_options1 = [
        "0", 'color protection', 'damage', 'curly enchancing', 'dandruff',
        'volumizing', 'anti aging', 'frizz', 'hold', 'thinning', 'oiliness',
        'straightening & smoothing', 'heat protection', 'shine'
    ]

    hconcerns_options2 = [
        "0", 'ColorProtection', 'Damage', 'CurlyEnhancing', 'Dandruff',
        'Volumizing', 'AntiAging', 'Frizz', 'Hold', 'Thinning', 'Oiliness',
        'StraighteningSmoothing', 'HeatProtection', 'Shine'
    ]
    
    if stone not in skintone_options:
        return "error"

    if stype not in skintype_options:
        return "error"

    if sconcerns not in sconcerns_options1:
        return "error"
    else:
        idx = sconcerns_options1.index(sconcerns)
        sconcerns = sconcerns_options2[idx]

    if ecolor not in ecolor_options:
        return "error"

    if hcolor not in hcolor_options:
        return "error"

    if hcondition not in hcondition_options1:
        return "error"
    else:
        idx = hcondition_options1.index(hcondition)
        hcondition = hcondition_options2[idx]

    if hconcerns not in hconcerns_options1:
        return "error"
    else:
        idx = hconcerns_options1.index(hconcerns)
        hconcerns = hconcerns_options2[idx]

    uid_query = ager + "_" + stone + "_" + stype + "_" + sconcerns + "_" + ecolor + "_" + hcolor + "_" + hcondition + "_" + hconcerns

    return uid_query


def get_products(cats=["0"], rel="0", gender="0"):

    relationship = [
        "mom", "girlfriend", "sister", "friend", "dad", "brother", "client",
        "boss", "family", "son", "daughter"
    ]
    
    if rel not in relationship:
        return "please reenter details (relationship)"

    products_to_consider = []

    men = pickle.load(open("for_men.pkl", "rb"))
    for_all = pickle.load(open("for_all.pkl", "rb"))
    hair_body = pickle.load(open("hair_body.pkl", "rb"))
    skincare = pickle.load(open("skincare.pkl", "rb"))
    makeup = pickle.load(open("makeup.pkl", "rb"))
    fragrance = pickle.load(open("fragrance.pkl", "rb"))

    products_to_consider = []
    
    if gender in ["they", "0"]:
        products_to_consider = men+hair_body+skincare+fragrance
    
    elif rel in ["friend", "dad", "brother", "boss", "colleague", "son", "client"] and gender=="he":
        products_to_consider = men
        
    elif rel in ["client", "boss", "colleague"] and gender!="he":
        products_to_consider = [i for i in for_all if i in men]
        
    else:
    
        if cats[0] == "0":
            products_to_consider = skincare + makeup + fragrance

        else:

            for cat in cats:
                if cat == "skincare":
                    products_to_consider.extend(skincare)
                elif cat == "makeup":
                    products_to_consider.extend(makeup)
                elif cat == "fragrance":
                    products_to_consider.extend(fragrance)

    products_to_consider = list(set(products_to_consider))

    return products_to_consider

In [9]:
def get_topn(uid, product_ids, n=50):

    algo = pickle.load(open("SVD_best.pkl", "rb"))

    items_to_pred = [(uid, iid, 0) for iid in product_ids]

    predictions = algo.test(items_to_pred)

    item_predictions = pd.DataFrame({
        "iid": product_ids,
        "est": [pred.est for pred in predictions]
    }).sort_values("est", ascending=False).reset_index(drop=True)

    return item_predictions.head(n)
